In [24]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.utils import all_estimators
from time import time
import scipy.stats as stats
from sklearn.utils.fixes import loguniform

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.linear_model import SGDClassifier

df = pd.read_csv(r'data/data_processed.csv')
df_cat = pd.read_csv(r'data/data_categorical.csv')

df_good = pd.concat([df_cat, df["MonthlyIncome"], df["BusinessTravel"], df["StockOptionLevel"], df["DistanceFromHome"]], axis=1)

# print(df_good)
# copy attrition and drop from dataframe
JAtt = df["Attrition"].values

df_good = df_good.drop(["Unnamed: 0"], axis=1)
df_good = df_good.drop(['Attrition'], axis=1)
# print(df_good)

# splitting new df
df_train = df_good.iloc[:1200,:]
df_val = df_good.iloc[1200:,:]
# splitting Jatt into new sizes
JAtt_train = JAtt[:1200]
JAtt_val = JAtt[1200:]

from sklearn.ensemble import AdaBoostClassifier


maxiter = np.ceil(10**6 / 1200)

# clf = AdaBoostClassifier(n_estimators=300, random_state=0, learning_rate=1)



In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_train = scaler.fit_transform(df_train)

In [18]:

clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=maxiter, tol=1e-5)

clf.fit(df_train, JAtt_train)

y_pred = clf.predict(df_val)

from sklearn import metrics

print(metrics.classification_report(JAtt_val, y_pred))
print(metrics.accuracy_score(JAtt_val, y_pred))


              precision    recall  f1-score   support

           0       0.82      1.00      0.90       221
           1       0.00      0.00      0.00        49

    accuracy                           0.82       270
   macro avg       0.41      0.50      0.45       270
weighted avg       0.67      0.82      0.74       270

0.8185185185185185


c:\Users\siman\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(
c:\Users\siman\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\siman\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\siman\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: 

In [19]:
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")

In [35]:
# specify parameters and distributions to sample from
param_dist = {
    "loss": ["hinge", "log_loss", "modified_huber", "squared_hinge", "perceptron", "squared_error", "huber", "epsilon_insensitive", "squared_epsilon_insensitive"],
    "average": [True, False],
    "penalty": ["l2", "l1", "elasticnet", None],
    "l1_ratio": stats.uniform(0, 1),
    "alpha": loguniform(1e-5, 1e0),
    "epsilon": loguniform(1e-3, 1e0),
    "eta0": stats.uniform(0, 1e4),
    "power_t": stats.uniform(-1e4, 1e4)
}

In [39]:
# run randomized search
n_iter_search = 1500
random_search = RandomizedSearchCV(
    clf, param_distributions=param_dist, n_iter=n_iter_search
)

In [41]:
start = time()
random_search.fit(df_train, JAtt_train)
print(
    "RandomizedSearchCV took %.2f seconds for %d candidates parameter settings."
    % ((time() - start), n_iter_search)
)
report(random_search.cv_results_)

# # use a full grid over all parameters
# param_grid = {
#     "average": [True, False],
#     "l1_ratio": np.linspace(0, 1, num=10),
#     "alpha": np.power(10, np.arange(-2, 1, dtype=float)),
# }

# # run grid search
# grid_search = GridSearchCV(clf, param_grid=param_grid)
# start = time()
# grid_search.fit(df_train, JAtt_train)

# print(
#     "GridSearchCV took %.2f seconds for %d candidate parameter settings."
#     % (time() - start, len(grid_search.cv_results_["params"]))
# )
# report(grid_search.cv_results_)

RandomizedSearchCV took 51.72 seconds for 1500 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.846 (std: 0.005)
Parameters: {'alpha': 0.01011128222257828, 'average': False, 'l1_ratio': 0.06358647885388169}

Model with rank: 1
Mean validation score: 0.846 (std: 0.006)
Parameters: {'alpha': 1.0789095893157956e-05, 'average': False, 'l1_ratio': 0.7890767127846355}

Model with rank: 1
Mean validation score: 0.846 (std: 0.006)
Parameters: {'alpha': 7.834258148017814e-05, 'average': False, 'l1_ratio': 0.6615298374022385}

Model with rank: 1
Mean validation score: 0.846 (std: 0.005)
Parameters: {'alpha': 0.37625075988644263, 'average': False, 'l1_ratio': 0.9041917378127025}

Model with rank: 1
Mean validation score: 0.846 (std: 0.005)
Parameters: {'alpha': 0.00010566945546559405, 'average': False, 'l1_ratio': 0.23167856279389754}

